In [1]:
import json
import os
import numpy as np
import pandas as pd
import re


In [2]:
os.chdir('/shared/0/projects/research-jam-summer-2024/')

In [4]:
mod_dir = 'data/english_only/prompting_results_clean/'
fname = os.listdir(mod_dir)[0] #there are 4 files here

# to get each dataframe
data = pd.read_json(mod_dir + fname, orient='records', lines=True)
vals = [c for c in data.columns if c.startswith('Prompt_')]
ids = [c for c in data.columns if c not in vals]
data = pd.melt(data, id_vars = ids, value_vars = vals, var_name = 'prompt', value_name = 'llm_turn_3')
data = data[data.llm_turn_3 != '[INVALID_DO_NOT_USE]']

# cross tab on no response
pd.crosstab((data['human_turn_3'] == '[no response]'), (data['llm_turn_3'] == '[no response]'))

# produce a score comparing human vs. llm text
#your_metric_scoring_function(data['human_turn_3'], data['llm_turn_3'])

llm_turn_3,False,True
human_turn_3,,
False,36493,1557
True,58772,3178


In [7]:
from scipy.spatial.distance import jensenshannon

from transformers import pipeline
from tqdm import tqdm
from scipy.spatial.distance import jensenshannon


def load_hf_model(model_name):
    cache_dir = None
    pipe = pipeline("text-classification", model=model_name, model_kwargs={"cache_dir": cache_dir},
                    device_map='cuda', max_length=512, truncation=True, return_all_scores=True)
    return pipe


def hf_data(data, max_char_len=2000):
    for row in data:
        yield row[:max_char_len]

def run_hf_model(data, model_name, column_name):
    pipe = load_hf_model(model_name)
    # First, human 3rd turn
    new_data = []
    for out in tqdm(pipe(hf_data(data), batch_size=4), total=len(data)):
        new_data.append([s['score'] for s in out])
    return new_data

def politeness(human, llm):
    human_politeness = run_hf_model(human, "Genius1237/xlm-roberta-large-tydip", 'politeness')#[politenessr.predict([text])[0] for text in human]
    llm_politeness = run_hf_model(llm, "Genius1237/xlm-roberta-large-tydip", 'politeness')#[politenessr.predict([text])[0] for text in llm]

    # Now all of the prompts
    return 1 - jensenshannon(human_politeness, llm_politeness, axis=1)#(np.array(human_politeness) - np.array(llm_politeness)).abs()


2024-07-26 00:42:06.580456: I tensorflow/core/platform/cpu_feature_guard.cc:181] Beginning TensorFlow 2.15, this package will be updated to install stock TensorFlow 2.15 alongside Intel's TensorFlow CPU extension plugin, which provides all the optimizations available in the package and more. If a compatible version of stock TensorFlow is present, only the extension will get installed. No changes to code or installation setup is needed as a result of this change.
More information on Intel's optimizations for TensorFlow, delivered as TensorFlow extension plugin can be viewed at https://github.com/intel/intel-extension-for-tensorflow.
2024-07-26 00:42:06.580533: I tensorflow/core/platform/cpu_feature_guard.cc:192] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
politeness = politeness(data['human_turn_3'], data['llm_turn_3'])


config.json:   0%|          | 0.00/827 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [ ]:
data.insert(len(data.columns), "metric_politeness", politeness)

In [ ]:
(data['human_turn_3'], data['llm_turn_3'])